In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
%aimport model

In [4]:
import skorch

In [5]:
import numpy as np

In [8]:
import keras

Using TensorFlow backend.


In [9]:
import torch

In [10]:
dataset = "fashion-mnist"

In [11]:
dataset_config = {
    'size': 32,
    'channels': 1,
    'classes': 10
}

In [12]:
model.Offnet(
    dataset,
    dataset_config['size'],
    dataset_config['channels'],
    dataset_config['classes'],
    total_block_number=6,
    widen_factor=5,
)

Offnet(
  (sequential): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): OffBlockGroup(
      (sequential): Sequential(
        (0): OffBlock(
          (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace)
          (conv1): FakeConv2d(
            (deform): Deform2d(
              (conv): Conv2d(16, 18, kernel_size=(1, 1), stride=(1, 1))
            )
            (conv): Conv2d(144, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (bn2): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace)
          (conv2): FakeConv2d(
            (deform): Deform2d(
              (conv): Conv2d(80, 18, kernel_size=(1, 1), stride=(1, 1))
            )
            (conv): Conv2d(720, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (conv_transform): Conv2d(16, 80, kernel_s

In [21]:
net = skorch.NeuralNetClassifier(
    module=model.Offnet(
        dataset,
        dataset_config['size'],
        dataset_config['channels'],
        dataset_config['classes'],
        total_block_number=6,
        widen_factor=5,
    ),
    criterion=torch.nn.CrossEntropyLoss,
    batch_size=32,
    max_epochs=30,
    optimizer=torch.optim.Adam,
    callbacks=[
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/untrainable_wrn{last_epoch[epoch]}.pt", monitor=None))
    ],
    device='cuda' if torch.cuda.is_available() else 'cpu',
    #train_split=None, # Workaround for https://github.com/dnouri/skorch/issues/218
    lr=1e-04,
)
net.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Offnet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): OffBlockGroup(
        (sequential): Sequential(
          (0): OffBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): FakeConv2d(
              (deform): Deform2d(
                (conv): Conv2d(16, 18, kernel_size=(1, 1), stride=(1, 1))
              )
              (conv): Conv2d(144, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
            )
            (bn2): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): FakeConv2d(
              (deform): Deform2d(
                (conv): Conv2d(80, 18, kernel_size=(1, 1), stride=(1, 1))
              )
              (conv): Conv2d(720, 80

In [22]:
def process(images):
    pad_images = np.pad(images, ((0, 0), (2, 2), (2, 2)), 'constant')
    number_of_images, height, width = pad_images.shape
    return pad_images.reshape(number_of_images, 1, height, width).astype(np.float32)  / 255.0

In [23]:
(x_train_bytes, y_train_bytes), (x_test_bytes, y_test_bytes) = keras.datasets.fashion_mnist.load_data()
x_train = process(x_train_bytes)
x_test = process(x_test_bytes)
y_train = y_train_bytes.astype(np.int64)
y_test = y_test_bytes.astype(np.int64)

In [ ]:
net.fit_loop(x_train, y_train)

In [ ]:
prediction_test = net.predict(x_test)

In [ ]:
(prediction_test == y_test).mean()